In [34]:
import json
import pdb
import os
import re

def get_tweet_text_from_json(file_path):
    events = []
    labels = []
    with open(file_path) as json_file:
#         pdb.set_trace()
        data = json.load(json_file)
        for key, value in data.items():
            events.append(value["text"])
            labels.append(value["label"])
        return events,labels
    
def get_tweet_text_from_json_unlabel(file_path):
    events = []
    with open(file_path) as json_file:
#         pdb.set_trace()
        data = json.load(json_file)
        for key, value in data.items():
            events.append(value["text"])
        return events
corinfo_train_events,corinfo_train_labels = get_tweet_text_from_json("train_negs.json")
misinfo_train_events,misinfo_train_labels = get_tweet_text_from_json("train.json")
train_events, train_labels = corinfo_train_events + misinfo_train_events,corinfo_train_labels + misinfo_train_labels
dev_events,dev_labels = get_tweet_text_from_json("dev.json")
test_unlabel = get_tweet_text_from_json_unlabel("test-unlabelled.json")

In [35]:
from nltk.corpus import stopwords
REPLACE_BY_SPACE_RE = re.compile('[/(){}\[\]\|@,;]')
BAD_SYMBOLS_RE = re.compile('[^0-9a-z #+_]')
STOPWORDS = set(stopwords.words('english'))

def clean_text(text):
    """
        text: a string
        
        return: modified initial string
    """
#     text = BeautifulSoup(text, "lxml").text # HTML decoding
#     pdb.set_trace()
    text = text.lower() # lowercase text
    text = REPLACE_BY_SPACE_RE.sub(' ', text) # replace REPLACE_BY_SPACE_RE symbols by space in text
    text = BAD_SYMBOLS_RE.sub('', text) # delete symbols which are in BAD_SYMBOLS_RE from text
    text = ' '.join(word for word in text.split() if word not in STOPWORDS) # delete stopwors from text
    return text
    
    
# df['post'] = df['post'].apply(clean_text)
# print_plot(10)

In [36]:
for i,event in enumerate(train_events):
    train_events[i] = clean_text(train_events[i])
for i,event in enumerate(dev_events):
    dev_events[i] = clean_text(dev_events[i])
for i,event in enumerate(test_unlabel):
    test_unlabel[i] = clean_text(test_unlabel[i])

In [37]:
def tokenize_text(text):
    tokens = []
    for sent in nltk.sent_tokenize(text, language='english'):
        for word in nltk.word_tokenize(sent, language='english'):
            if len(word) < 2:
                continue
            tokens.append(word)
    return tokens

In [38]:
import nltk
train_tokenized = []
dev_tokenized = []
test_tokenized = []
for event in train_events:
    train_tokenized.append(tokenize_text(event))
for event in dev_events:
    dev_tokenized.append(tokenize_text(event))
for event in test_unlabel:
    test_tokenized.append(tokenize_text(event))  

In [6]:
#########################get negtive labeled data#########################

#encoding:'utf-8'
import urllib.request
from bs4 import BeautifulSoup
import os
import time
import codecs
import json

In [23]:
import time
import re

def get_articles(url):
    header={'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"}
    ret=urllib.request.Request(url=url,headers=header)
    res=urllib.request.urlopen(ret)
    response=BeautifulSoup(res,'html.parser')
    datas=response.find_all('p')
    dr = re.compile(r'<[^>]+>',re.S)
    datas_clean = dr.sub('',str(datas))
#     pdb.set_trace()
    return datas_clean.replace('\n','').replace('[','').replace(']','').replace('(','').replace(')','')

def getDatas():
    header={'User-Agent':"Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_3) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/80.0.3987.163 Safari/537.36"}    
    url = "https://www.climatechangeinaustralia.gov.au/en/climate-campus/climate-system/"
    ret=urllib.request.Request(url=url,headers=header)
    res=urllib.request.urlopen(ret)
    response=BeautifulSoup(res,'html.parser')
    datas=response.find_all(lambda tag: tag.name == 'ul' and tag.get('class') == ['nav_ul'])
    
    regex_href = re.compile('<a href="(.*?)">')
    nav_urls = re.findall(regex_href, str(datas))
    
    train_negs = {}
    for i,nav_url in enumerate(nav_urls):
        dic_train = {}
        dic_train['text'] = get_articles('https://www.climatechangeinaustralia.gov.au' + nav_url)
        dic_train['label'] = 0
        train_negs['train-%s'%i] = dic_train
    jstr = json.dumps(train_negs,ensure_ascii = False)
    with open('train_negs.json','w') as f:
        f.write(jstr)
#     make a folder
#     folder_name="output"
#     if not os.path.exists(folder_name):
#         os.mkdir(folder_name)
    pass
getDatas()